# Image to Text

In [1]:
!export PYTHONPATH=":./python"

from devtools import debug  # noqa: E402

In [9]:
from python.ai_core.llm import get_llm
from python.ai_core.prompts import def_prompt

llm = get_llm(llm_id="gpt_4o_openai")  # Select an LLM that supports Vision
# llm = get_llm(llm_id="gpt_4_edenai")  # Select an LLM that supports Vision

2024-06-27 23:38:26.386 | INFO     | python.ai_core.llm:get_llm:395 - get LLM : gpt_4o_openai - configurable: True
2024-06-27 23:38:26.831 | INFO     | python.ai_core.llm:get_configurable:351 - Cannot load gemini_pro_google: No module named 'langchain_google_vertexai'


In [10]:
import base64

IMAGE_PATH = "use_case_data/railway/network rail.png"


# Open the image file and encode it as a base64 string
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


base64_image = encode_image(IMAGE_PATH)

messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant that responds in Markdown.",
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "Describe the images as an alternative text"},
            {
                "type": "image_url",
                "image_url": {"url": f"data:image/png;base64,{base64_image}"},
            },
        ],
    },
]

r = llm.invoke(messages)
print(r)

content='```markdown\nA schematic diagram of Doncaster Junction railway signaling layout. The diagram shows various tracks, signals, and points with labels indicating distances from the signal box. Key elements include:\n\n- Multiple tracks labeled "Down Main" and "Up Main."\n- Signals numbered 1, 2, 3, 5, 6, 12, 13, 23, 24, 25, 26, 27, 28, 31, 34, 35, and 36.\n- Points and crossings labeled with numbers corresponding to the signals.\n- Distances from the signal box are marked, such as "1203 YDS. FROM BOX," "300 YDS. FROM BOX," "171 YDS. FROM BOX," and so on.\n- A note indicating "7 ELECTRIC RELEASE TO 2 LEVER GROUND FRAME."\n- A section labeled "PIPERING LANE CROSSING."\n- A list of spare levers numbered 1, 4, 8, 14, 15, 16, 17, 18, 19, 20, 21, 22, 29, 32, and 33.\n- The diagram is bordered and titled "DONCASTER JUNCTION" at the top.\n```\n' response_metadata={'token_usage': {'completion_tokens': 263, 'prompt_tokens': 1473, 'total_tokens': 1736}, 'model_name': 'gpt-4o-2024-05-13', 'sy